In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
"""
This file just to save some functions to help keep the notebooks clean.  
"""
import cv2
import pandas as pd
import os
import numpy as np
import lungs_finder as lf

<IPython.core.display.Javascript object>

In [3]:
def process_data(img_path, IMAGE_RESOLUTION, BORDER):
    """
    This function wil get the image path, load it and then return it as object.

    :img_path string: The image path.
    :IMAGE_RESOLUTION tuple: Tuple to define the image resolution (x, y, z).
    :BORDER int: The pixel that will be used to cut the image.
    """

    try:
        # Read the image
        img = cv2.imread(img_path)
        # Resize the image
        img = cv2.resize(img, IMAGE_RESOLUTION[:2])

        # Transform the color to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Identify lungs
        img = lf.get_lungs(img, padding=0)

        # Scale the image between zero and one
        img = img / 255.0

        # Resize the image
        img = cv2.resize(img, IMAGE_RESOLUTION[:2])

        # Reshape the image
        img = np.reshape(img, IMAGE_RESOLUTION)

        return img
    except:
        # Read the image again
        img = cv2.imread(img_path)
        # Resize the image
        img = cv2.resize(img, IMAGE_RESOLUTION[:2])

        # Transform the color to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Scale the image between zero and one
        img = img / 255.0

        # Cut the image
        img = img[
            BORDER : IMAGE_RESOLUTION[0] - BORDER, BORDER : IMAGE_RESOLUTION[0] - BORDER
        ]

        # Resize the image
        img = cv2.resize(img, IMAGE_RESOLUTION[:2])

        # Reshape the image
        img = np.reshape(img, IMAGE_RESOLUTION)

        return img



<IPython.core.display.Javascript object>

# Load raw dataset

In [4]:
dataset = pd.read_csv("data_image.csv")
dataset.head()

,image_name,full_path_raw,full_path_transformed,kind,flag
0,IM-0115-0001.jpeg,raw\train\NORMAL\IM-0115-0001.jpeg,transformed\train\NORMAL\IM-0115-0001.jpeg,train,0
1,IM-0117-0001.jpeg,raw\train\NORMAL\IM-0117-0001.jpeg,transformed\train\NORMAL\IM-0117-0001.jpeg,train,0
2,IM-0119-0001.jpeg,raw\train\NORMAL\IM-0119-0001.jpeg,transformed\train\NORMAL\IM-0119-0001.jpeg,train,0
3,IM-0122-0001.jpeg,raw\train\NORMAL\IM-0122-0001.jpeg,transformed\train\NORMAL\IM-0122-0001.jpeg,train,0
4,IM-0125-0001.jpeg,raw\train\NORMAL\IM-0125-0001.jpeg,transformed\train\NORMAL\IM-0125-0001.jpeg,train,0


<IPython.core.display.Javascript object>

In [5]:
raw = 'raw'
transformed = 'transformed'

path_train_normal = os.path.join("train","NORMAL")
path_train_pneu = os.path.join("train","PNEUMONIA")

path_test_normal = os.path.join("test","NORMAL")
path_test_pneu = os.path.join("test","PNEUMONIA")

path_val_normal = os.path.join("val","NORMAL")
path_val_pneu =  os.path.join("val","PNEUMONIA")


<IPython.core.display.Javascript object>

In [6]:
from pathlib import Path

Path(os.path.join(transformed, path_train_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_train_pneu)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_test_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_test_pneu)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_val_normal)).mkdir(parents=True, exist_ok=True)
Path(os.path.join(transformed, path_val_pneu)).mkdir(parents=True, exist_ok=True)

<IPython.core.display.Javascript object>

In [7]:
%%time
from joblib import Parallel, delayed


IMAGE_RESOLUTION = (250, 250, 1)
BORDER = 30
print('Image shape: {}'.format(IMAGE_RESOLUTION))

images_transformed = Parallel(n_jobs=4)(delayed(process_data)(img_path, IMAGE_RESOLUTION, BORDER) for img_path in dataset.full_path_raw)

Image shape: (250, 250, 1)
Wall time: 1min 32s


<IPython.core.display.Javascript object>

In [8]:
%%time
results = Parallel(n_jobs=4)(delayed(cv2.imwrite)(img_path_transformed, images_transformed[index]*255) for index, img_path_transformed in enumerate(dataset.full_path_transformed))

if sum(results) == len(dataset.full_path_raw):
    print("All images were transformed!")

All images were transformed!
Wall time: 13.9 s


<IPython.core.display.Javascript object>

In [9]:
# Firebase Connection

<IPython.core.display.Javascript object>

In [39]:
!pip3 install tqdm

<IPython.core.display.Javascript object>

In [43]:
import pyrebase
from firebase_env import firebase_config
from tqdm.notebook import tqdm


firebase = pyrebase.initialize_app(firebase_config)
storage = firebase.storage()

<IPython.core.display.Javascript object>

In [46]:
def save_image_firebase(image_path, storage):
    try:
        storage.child(image_path).put(image_path)
        return True
    except:
        return False


for img_path_transformed in tqdm(dataset.full_path_transformed):
    # To linux
    aux = img_path_transformed.replace("\\", "/")
    save_image_firebase(aux, storage)

<IPython.core.display.Javascript object>